In [3]:
pip install tensorflow matplotlib


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow flask opencv-python numpy pillow


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Dataset folder inside your workspace
dataset_dir = 'rice_leaf_diseases'  # Make sure this matches your folder name

# Check if dataset exists
if not os.path.exists(dataset_dir):
    raise FileNotFoundError(f"Dataset folder not found at: {dataset_dir}")

# Create train/ and val/ folders in your workspace
train_dir = 'train'
val_dir = 'val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get the class folders
classes = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
print(f"Classes found: {classes}")

# Create subfolders for each class inside train/ and val/
for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

# Split the images into train and val
for class_name in classes:
    class_path = os.path.join(dataset_dir, class_name)
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Copy images to train/
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    
    # Copy images to val/
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

Classes found: ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os


In [8]:
# Directories
train_dir = 'train'
val_dir = 'val'

# Image data generators with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse'  # because labels are integers
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse'
)

# Check classes
print("Class indices:", train_data.class_indices)


Found 96 images belonging to 3 classes.


Found 24 images belonging to 3 classes.
Class indices: {'Bacterial leaf blight': 0, 'Brown spot': 1, 'Leaf smut': 2}


In [9]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_data.class_indices), activation='softmax')  # number of classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


c:\Users\nandi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    train_data,
    epochs=10,  # increase if dataset is larger
    validation_data=val_data
)

# Save the trained model
model.save('rice_leaf_disease_model.h5')


c:\Users\nandi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2689 - loss: 6.0538 - val_accuracy: 0.3333 - val_loss: 1.1537
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.3286 - loss: 1.1071 - val_accuracy: 0.3333 - val_loss: 1.2376
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.2647 - loss: 1.1361 - val_accuracy: 0.5417 - val_loss: 1.0571
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5624 - loss: 1.0595 - val_accuracy: 0.3750 - val_loss: 1.0499
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.3680 - loss: 1.0579 - val_accuracy: 0.5000 - val_loss: 0.9885
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5058 - loss: 1.0068 - val_accuracy: 0.5417 - val_loss: 0.9029
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5461 - loss: 0.8830 - val_accuracy: 0.6667 - val_loss: 0.8298
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7574 - loss: 0.7249 - val_accuracy: 0.4583 - val_loss: 0.8629
Epoch 9/

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load model
model = tf.keras.models.load_model('rice_leaf_disease_model.h5')

# Define class labels
class_labels = list(train_data.class_indices.keys())

# Load image
img_path = 'rice_leaf_diseases/Bacterial leaf blight/DSC_0365.JPG'  # replace with your image
img = image.load_img(img_path, target_size=(256,256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Predict
pred = model.predict(img_array)
predicted_class = class_labels[np.argmax(pred)]
print("Disease detected:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Disease detected: Bacterial leaf blight


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('rice_leaf_disease_model.h5')

# Define class labels
class_labels = ['Bacterial_Leaf_Blight', 'Brown_Spot', 'Leaf_Smut', 'Tungro', 'Healthy_Leaf']

# Fertilizer / treatment recommendations (expanded)
fertilizer_recommendations = {
    'Bacterial_Leaf_Blight': (
        "💧 Water management: Avoid excessive irrigation.\n"
        "💊 Fungicide: Use Copper-based fungicides or Streptomycin sprays.\n"
        "🌱 Fertilizer: Balanced NPK; avoid excess nitrogen which favors disease.\n"
        "🧹 Field sanitation: Remove infected leaves to reduce spread."
    ),
    'Brown_Spot': (
        "💊 Fungicide: Apply Mancozeb or Copper Oxychloride.\n"
        "🌱 Fertilizer: Use Potassium-rich fertilizers (K2O) to improve plant resistance.\n"
        "⚖️ Avoid excess nitrogen which can worsen the disease.\n"
        "🧹 Remove infected residues after harvest."
    ),
    'Leaf_Smut': (
        "💊 Fungicide: Systemic fungicides like Propiconazole or Carbendazim.\n"
        "🌱 Fertilizer: Balanced NPK, slightly higher Potassium.\n"
        "🌾 Crop rotation: Avoid planting rice continuously in the same field.\n"
        "🧹 Maintain proper spacing to reduce humidity."
    ),
    'Tungro': (
        "🦟 Insect control: Control leafhopper vector using insecticides.\n"
        "🌱 Fertilizer: Balanced NPK; Zinc application can improve plant tolerance.\n"
        "🧹 Remove diseased plants to reduce virus spread.\n"
        "💧 Avoid excessive water logging."
    ),
    'Healthy_Leaf': (
        "✅ Plant is healthy.\n"
        "🌱 Maintain balanced NPK fertilization.\n"
        "💧 Follow proper irrigation.\n"
        "🧹 Continue regular monitoring to prevent disease."
    )
}

# Load the image
img_path = 'rice_leaf_diseases\Brown spot\DSC_0106.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Predict
pred = model.predict(img_array)
predicted_class = class_labels[np.argmax(pred)]

# Display results
print("Disease detected:", predicted_class)
print("Recommended fertilizer/treatment:")
print(fertilizer_recommendations[predicted_class])


<>:46: SyntaxWarning: invalid escape sequence '\B'
<>:46: SyntaxWarning: invalid escape sequence '\B'
C:\Users\nandi\AppData\Local\Temp\ipykernel_5544\1319422732.py:46: SyntaxWarning: invalid escape sequence '\B'
  img_path = 'rice_leaf_diseases\Brown spot\DSC_0106.jpg'  # Replace with your image path


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Disease detected: Leaf_Smut
Recommended fertilizer/treatment:
💊 Fungicide: Systemic fungicides like Propiconazole or Carbendazim.
🌱 Fertilizer: Balanced NPK, slightly higher Potassium.
🌾 Crop rotation: Avoid planting rice continuously in the same field.
🧹 Maintain proper spacing to reduce humidity.


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# ------------------------------
# Set your local image path here
img_path = r"C:\Users\nandi\OneDrive\Desktop\bacterialoozedry.jpg"
# ------------------------------

# Load the trained model
model = tf.keras.models.load_model('rice_leaf_disease_model.h5')

# Define class labels
class_labels = ['Bacterial_Leaf_Blight', 'Brown_Spot', 'Leaf_Smut', 'Tungro', 'Healthy_Leaf']

# Fertilizer / treatment recommendations
fertilizer_recommendations = {
    'Bacterial_Leaf_Blight': (
        "💧 Water management: Avoid excessive irrigation.\n"
        "💊 Fungicide: Use Copper-based fungicides or Streptomycin sprays.\n"
        "🌱 Fertilizer: Balanced NPK; avoid excess nitrogen which favors disease.\n"
        "🧹 Field sanitation: Remove infected leaves to reduce spread."
    ),
    'Brown_Spot': (
        "💊 Fungicide: Apply Mancozeb or Copper Oxychloride.\n"
        "🌱 Fertilizer: Use Potassium-rich fertilizers (K2O) to improve plant resistance.\n"
        "⚖️ Avoid excess nitrogen which can worsen the disease.\n"
        "🧹 Remove infected residues after harvest."
    ),
    'Leaf_Smut': (
        "💊 Fungicide: Systemic fungicides like Propiconazole or Carbendazim.\n"
        "🌱 Fertilizer: Balanced NPK, slightly higher Potassium.\n"
        "🌾 Crop rotation: Avoid planting rice continuously in the same field.\n"
        "🧹 Maintain proper spacing to reduce humidity."
    ),
    'Tungro': (
        "🦟 Insect control: Control leafhopper vector using insecticides.\n"
        "🌱 Fertilizer: Balanced NPK; Zinc application can improve plant tolerance.\n"
        "🧹 Remove diseased plants to reduce virus spread.\n"
        "💧 Avoid excessive water logging."
    ),
    'Healthy_Leaf': (
        "✅ Plant is healthy.\n"
        "🌱 Maintain balanced NPK fertilization.\n"
        "💧 Follow proper irrigation.\n"
        "🧹 Continue regular monitoring to prevent disease."
    )
}

# Load the image
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Predict
pred = model.predict(img_array)
predicted_class = class_labels[np.argmax(pred)]

# Display results
print(f"\nImage selected: {img_path}")
print("Disease detected:", predicted_class)
print("Recommended fertilizer/treatment:")
print(fertilizer_recommendations[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step

Image selected: C:\Users\nandi\OneDrive\Desktop\bacterialoozedry.jpg
Disease detected: Leaf_Smut
Recommended fertilizer/treatment:
💊 Fungicide: Systemic fungicides like Propiconazole or Carbendazim.
🌱 Fertilizer: Balanced NPK, slightly higher Potassium.
🌾 Crop rotation: Avoid planting rice continuously in the same field.
🧹 Maintain proper spacing to reduce humidity.


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# ------------------------------
# Set your local image path here
img_path = r"C:\Users\nandi\OneDrive\Desktop\nannnn\rice_leaf_diseases\Brown spot\DSC_0104.jpg"
# ------------------------------

# Load the trained model
model = tf.keras.models.load_model('rice_leaf_disease_model.h5')

# Define class labels
class_labels = ['Bacterial_Leaf_Blight', 'Brown_Spot', 'Leaf_Smut', 'Tungro', 'Healthy_Leaf']

# Fertilizer / treatment recommendations
fertilizer_recommendations = {
    'Bacterial_Leaf_Blight': (
        "💧 Water management: Avoid excessive irrigation.\n"
        "💊 Fungicide: Use Copper-based fungicides or Streptomycin sprays.\n"
        "🌱 Fertilizer: Balanced NPK; avoid excess nitrogen which favors disease.\n"
        "🧹 Field sanitation: Remove infected leaves to reduce spread."
    ),
    'Brown_Spot': (
        "💊 Fungicide: Apply Mancozeb or Copper Oxychloride.\n"
        "🌱 Fertilizer: Use Potassium-rich fertilizers (K2O) to improve plant resistance.\n"
        "⚖️ Avoid excess nitrogen which can worsen the disease.\n"
        "🧹 Remove infected residues after harvest."
    ),
    'Leaf_Smut': (
        "💊 Fungicide: Systemic fungicides like Propiconazole or Carbendazim.\n"
        "🌱 Fertilizer: Balanced NPK, slightly higher Potassium.\n"
        "🌾 Crop rotation: Avoid planting rice continuously in the same field.\n"
        "🧹 Maintain proper spacing to reduce humidity."
    ),
    'Tungro': (
        "🦟 Insect control: Control leafhopper vector using insecticides.\n"
        "🌱 Fertilizer: Balanced NPK; Zinc application can improve plant tolerance.\n"
        "🧹 Remove diseased plants to reduce virus spread.\n"
        "💧 Avoid excessive water logging."
    ),
    'Healthy_Leaf': (
        "✅ Plant is healthy.\n"
        "🌱 Maintain balanced NPK fertilization.\n"
        "💧 Follow proper irrigation.\n"
        "🧹 Continue regular monitoring to prevent disease."
    )
}

# Load and preprocess the image
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Predict disease
pred = model.predict(img_array)
predicted_class = class_labels[np.argmax(pred)]

# Display results
print(f"\nImage selected: {img_path}")
print("Disease detected:", predicted_class)
print("Recommended fertilizer/treatment:")
print(fertilizer_recommendations[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step

Image selected: C:\Users\nandi\OneDrive\Desktop\nannnn\rice_leaf_diseases\Brown spot\DSC_0104.jpg
Disease detected: Brown_Spot
Recommended fertilizer/treatment:
💊 Fungicide: Apply Mancozeb or Copper Oxychloride.
🌱 Fertilizer: Use Potassium-rich fertilizers (K2O) to improve plant resistance.
⚖️ Avoid excess nitrogen which can worsen the disease.
🧹 Remove infected residues after harvest.


In [15]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install scikit-learn opencv-python pillow numpy streamlit joblib


Note: you may need to restart the kernel to use updated packages.
